<a href="https://colab.research.google.com/github/Darkdew/SVMGridSearchCV/blob/master/SVMGridSearchCV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [0]:
df= pd.read_csv('/content/drive/My Drive/Colab Notebooks/SVMGridSearchCv/Advertising_data.csv')

In [3]:
df.head()

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19.0,19000.0,0
1,15810944,Male,35.0,20000.0,0
2,15668575,Female,26.0,43000.0,0
3,15603246,Female,27.0,57000.0,0
4,15804002,Male,19.0,76000.0,0


In [4]:
df.isnull().sum().any()

False

In [0]:
X= df.iloc[:,[2,3] ].values

In [10]:
type(X)

numpy.ndarray

In [0]:
y= df.iloc[:, 4].values

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size=0.2, random_state=7)

In [0]:
#Feature Scaling

from sklearn.preprocessing import StandardScaler
ss= StandardScaler()
X_train= ss.fit_transform(X_train)
X_test= ss.transform(X_test)

In [18]:
#Lets fit Kernel SVM to our training dataset
#there are 2 kinds of kernels using which SVM classifies- linear and rbf
#linear is used when we think that the separation between the 2 classes can be done linearly
#rbf otherwise
from sklearn.svm import SVC
SVMclassifier= SVC(kernel='linear', random_state=0)
SVMclassifier.fit(X_train, y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=0, shrinking=True, tol=0.001,
    verbose=False)

In [0]:
y_pred= SVMclassifier.predict(X_test)

In [20]:
#Model Evaluation
from sklearn.metrics import confusion_matrix, accuracy_score

cm= confusion_matrix(y_test, y_pred)
score= accuracy_score(y_test, y_pred)
print(cm)
print(score)

[[50  4]
 [ 5 21]]
0.8875


In [0]:
#though we applied linear kernel how can we be so sure that our classes could be linearly seprated
#also, there are many parameters inside the SVM Classifier
#how to use the optimal values for each of the parameters?
#this is where GridSearchCV helps

from sklearn.model_selection import GridSearchCV

parameters= [{'C': [1,10,100,1000], 'kernel': ['linear']},
             
             {'C': [1,10,100,1000], 'kernel': ['rbf'], 'gamma': [0.1, 0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]}
             ]

grid_search= GridSearchCV(estimator= SVMclassifier, 
                          param_grid= parameters,
                          scoring= 'accuracy',
                          cv=10,
                          n_jobs=-1 #use all CPU cores
                          )

In [0]:
grid_search_model= grid_search.fit(X_train, y_train)

In [24]:
grid_search_model.best_params_

{'C': 1, 'gamma': 0.8, 'kernel': 'rbf'}

In [25]:
grid_search_model.best_score_ #Note: This is GridSearch's best score: not of our model

0.915625

In [27]:
#Now that we have the best parameters at hand found out by GridSearchCV
#lets apply them in our SVM model

#So the final SVM Model Accuracy can be calculated as

from sklearn.svm import SVC

classifier= SVC(gamma= 0.8)
classifier.fit(X_train, y_train)

y_pred= classifier.predict(X_test)
score= accuracy_score(y_test, y_pred)
score

0.9375